<a href="https://colab.research.google.com/github/ciertou/with-linux/blob/main/activation_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("i love you so much!")

i love you so much!


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# Load model and tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, output_hidden_states=True)
#model.to("cpu")
model.eval()

# Choose the layer to extract activations from
target_layer = 6  # Adjust based on model architecture


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Choose the layer to extract activations from
target_layer = 6  # Adjust based on model architecture

# Sample prompts
enthusiastic_prompts = [
    "I'm so excited about the new product launch!",
    "What a fantastic day we're having!",
    "I can't wait to try this amazing new feature!"
]

unenthusiastic_prompts = [
    "The new product launch happened.",
    "It's just another day.",
    "There's a new feature, I guess."
]

def get_hidden_representation(prompt, model, tokenizer, target_layer):
    inputs = tokenizer(prompt, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
        hidden_states = outputs.hidden_states  # Tuple: (layer0, layer1, ..., layerN)
    # Use the hidden state from the target layer for the last token
    return hidden_states[target_layer][0, -1]  # Shape: (hidden_size,)

def average_representation(prompts, model, tokenizer, target_layer):
    vecs = [get_hidden_representation(p, model, tokenizer, target_layer) for p in prompts]
    return torch.stack(vecs).mean(dim=0)

In [ ]:
# Step 1: Get average vectors
E = average_representation(enthusiastic_prompts, model, tokenizer, target_layer)
UE = average_representation(unenthusiastic_prompts, model, tokenizer, target_layer)

# Step 2: Compute steering vector
#Apply alpha to scale the steering vector
alpha = 0.5  # <-- Adjust this value to control influence
steering_vector = alpha * (E - UE)

In [ ]:
print(f"printing {UE}")
print(f"printing {E}")

In [ ]:


# Step 3: Modify inference with steering
def generate_with_steering(prompt, model, tokenizer, steering_vector, target_layer, max_new_tokens=20):
    inputs = tokenizer(prompt, return_tensors='pt')
    input_ids = inputs['input_ids']
    generated = input_ids.clone()

    for _ in range(max_new_tokens):
        with torch.no_grad():
            outputs = model(input_ids=generated, output_hidden_states=True)
            hidden_states = list(outputs.hidden_states)
            # Apply the steering vector at the target layer, last token only
            hidden_states[target_layer][0, -1] += steering_vector

            # Reconstruct logits from the modified hidden state
            next_token_logits = model.lm_head(hidden_states[-1][:, -1, :])  # (1, vocab_size)

            next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
            generated = torch.cat((generated, next_token), dim=1)

    return tokenizer.decode(generated[0])


In [ ]:
# Test it
prompt = "Today we are launching our new product"
output = generate_with_steering(prompt, model, tokenizer, steering_vector, target_layer)
print(output)